In [1]:
from sklearn.model_selection import GridSearchCV
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


In [2]:

# Load the dataset
file_path = 'Data\Chicago 2021-07-01 to 2024-03-17.csv'
data = pd.read_csv(file_path)

# Data Preprocessing
data['datetime'] = pd.to_datetime(data['datetime'])
data['Year'] = data['datetime'].dt.year
data['Month'] = data['datetime'].dt.month
data['Day'] = data['datetime'].dt.day
data['DayOfYear'] = data['datetime'].dt.dayofyear
data['WeekOfYear'] = data['datetime'].dt.isocalendar().week
data['windgust'] = data['windgust'].fillna(0)  # Fill missing wind gusts with 0

# Feature selection
features_to_include = ['humidity', 'windgust',
                       'cloudcover', 'tempmin', 'tempmax']
X = data[['Year', 'Month', 'Day', 'DayOfYear',
          'WeekOfYear'] + features_to_include]
y = data['tempmax']

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

# Define and train the model
optimized_model = XGBRegressor(
    n_estimators=656,
    max_depth=5,
    learning_rate=0.24057515827045411,
    colsample_bytree=0.9500983814768778,
    subsample=0.27848270266263386,
    reg_alpha=0.3728777359951313,
    reg_lambda=1.9881262022259722,
    random_state=42
)
optimized_model.fit(X_train, y_train, eval_set=[
                    (X_val, y_val)], early_stopping_rounds=50, verbose=False)



c:\Users\arune\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9500983814768778, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.24057515827045411, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=656, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [6]:
# Prepare today's data for prediction
from datetime import datetime
# Assign 'today' to a specific date, for example, March 14, 2024
today = datetime(year=2024, month=4, day=1)
# today = datetime.now()
df_today = pd.DataFrame({
    'Year': [today.year],
    'Month': [today.month],
    'Day': [today.day],
    'DayOfYear': [today.timetuple().tm_yday],
    'WeekOfYear': [today.isocalendar()[1]],
    'humidity': 74.6,  # Placeholder, replace with actual data if available
    'windgust': 21.9,       # Assuming missing wind gusts are 0
    # Placeholder, replace with actual data if available
    'cloudcover': 90.9,
    # Placeholder, replace with actual data if available
    'tempmin': 39.2,
    # Placeholder, it's what we're predicting but needs to be in the input
    'tempmax': 50.4
})

# Fill missing values for `df_today` as per your dataset's requirements
# For example, df_today['humidity'].fillna(value, inplace=True)

# Scale df_today using the same scaler used for the training data
X_today_scaled = scaler.transform(df_today)

# Predict the max temperature for today
today_max_temp_pred = optimized_model.predict(X_today_scaled)
print(
    f'Predicted max temperature for today: {round(today_max_temp_pred[0])}°F')

Predicted max temperature for today: 50°F
